In [1]:
import pandas as pd
data=pd.read_csv('NER dataset.csv',encoding='latin-1')

In [2]:
data.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [3]:
data.shape

(1048575, 4)

In [4]:
data.isnull().sum()

Sentence #    1000616
Word               10
POS                 0
Tag                 0
dtype: int64

In [5]:
data.ffill(inplace=True)

In [6]:
data.isnull().sum()

Sentence #    0
Word          0
POS           0
Tag           0
dtype: int64

In [7]:
class getsentences(object):
    def __init__(self,data):
        self.n_words=1.0
        self.data=data
        self.empty=False
        agg_fun=lambda s:[(w,t)for w,t,p in zip(s['Word'].values.tolist(),s['POS'].values.tolist(),s['Tag'].values.tolist())]
        self.grouped=self.data.groupby('Sentence #').apply(agg_fun)
        self.sentence=[s for s in self.grouped]
getter=getsentences(data)
sentence=getter.sentence
print(sentence[0])
len(sentence)
      

[('Thousands', 'NNS'), ('of', 'IN'), ('demonstrators', 'NNS'), ('have', 'VBP'), ('marched', 'VBN'), ('through', 'IN'), ('London', 'NNP'), ('to', 'TO'), ('protest', 'VB'), ('the', 'DT'), ('war', 'NN'), ('in', 'IN'), ('Iraq', 'NNP'), ('and', 'CC'), ('demand', 'VB'), ('the', 'DT'), ('withdrawal', 'NN'), ('of', 'IN'), ('British', 'JJ'), ('troops', 'NNS'), ('from', 'IN'), ('that', 'DT'), ('country', 'NN'), ('.', '.')]


47959

In [8]:
x=[]
y=[]
for sen in sentence:
    x_sent=[]
    y_sent=[]
    for entity in sen:
        x_sent.append(entity[0])
        y_sent.append(entity[1])
    x.append(x_sent)
    y.append(y_sent)

In [9]:
num_word=len(set([word.lower()  for sentence in x for word in sentence]))
num_tag=len(set([word.lower()  for sentence in y for word in sentence]))

In [10]:
print("Total number of tagged sentences: {}".format(len(x)))
print("Vocabulary size: {}".format(num_word))
print("Total number of tags: {}".format(num_tag))



Total number of tagged sentences: 47959
Vocabulary size: 31817
Total number of tags: 42


In [12]:
import tensorflow  
from tensorflow.keras.preprocessing.text import Tokenizer
word_tokenizer=Tokenizer()
word_tokenizer.fit_on_texts(x)
x_encode=word_tokenizer.texts_to_sequences(x)

In [13]:
tag_tokenizer=Tokenizer()
tag_tokenizer.fit_on_texts(y)
y_encoded=tag_tokenizer.texts_to_sequences(y)

In [14]:
dif_length=[1 if len(input)!=len(output) else 0 for input,output in zip(x_encode,y_encoded)]
print("{} sentences have disparate input-output lengths.".format(sum(dif_length)))


0 sentences have disparate input-output lengths.


In [15]:
length=[len(x) for x in x_encode]
print(max(length))

104


In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length=104
x_pad=pad_sequences(x_encode,maxlen=max_length,padding="pre")
y_pad=pad_sequences(y_encoded,maxlen=max_length,padding="pre")

In [17]:
x,y=x_pad,y_pad

In [18]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

In [21]:
y.shape[2]

43

In [22]:
vocab_size=len(word_tokenizer.word_index)+1
vocab_size

31818

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,TimeDistributed,Dense
vocab_size=len(word_tokenizer.word_index)+1
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=100,input_length=length))
model.add(SimpleRNN(64,return_sequences=True))
model.add(TimeDistributed(Dense(43,activation='sigmoid')))


C:\Users\vj75v\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
rnn_training = model.fit(x_train, y_train, batch_size=128, epochs=4, validation_data=(x_test, y_test))


Epoch 1/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 100s 271ms/step - accuracy: 0.8313 - loss: 1.4787 - val_accuracy: 0.9665 - val_loss: 0.2005
Epoch 2/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 95s 281ms/step - accuracy: 0.9756 - loss: 0.1494 - val_accuracy: 0.9862 - val_loss: 0.0740
Epoch 3/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 268ms/step - accuracy: 0.9893 - loss: 0.0594 - val_accuracy: 0.9892 - val_loss: 0.0473
Epoch 4/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 90s 266ms/step - accuracy: 0.9918 - loss: 0.0375 - val_accuracy: 0.9901 - val_loss: 0.0378


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,TimeDistributed,Dense
vocab_size=len(word_tokenizer.word_index)+1
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=100,input_length=length))
model.add(LSTM(64,return_sequences=True))
model.add(TimeDistributed(Dense(43,activation='sigmoid')))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
rnn_training = model.fit(x_train, y_train, batch_size=128, epochs=4, validation_data=(x_test, y_test))


Epoch 1/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 125s 343ms/step - accuracy: 0.8036 - loss: 1.4828 - val_accuracy: 0.9611 - val_loss: 0.2364
Epoch 2/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 120s 355ms/step - accuracy: 0.9717 - loss: 0.1757 - val_accuracy: 0.9851 - val_loss: 0.0858
Epoch 3/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 112s 331ms/step - accuracy: 0.9882 - loss: 0.0702 - val_accuracy: 0.9884 - val_loss: 0.0541
Epoch 4/4
338/338 ━━━━━━━━━━━━━━━━━━━━ 103s 306ms/step - accuracy: 0.9911 - loss: 0.0440 - val_accuracy: 0.9896 - val_loss: 0.0424
